Feature engineering

In [34]:
import pandas as pd
import re
import pickle

In [35]:
data = pickle.load(open('../temp/historic-clean-2021-03-15.pkl', 'rb'))

In [36]:
data

,inspection_id,facility_type,risk,zip,inspection_date,inspection_type,results,violations,latitude,longitude,num_violations
0,2492768,restaurant,risk_2_medium,60647,2021-03-12,license,pass,na,41.917135,-87.687544,0
1,2492746,daycare,risk_1_high,60624,2021-03-11,license,not_pass,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.885920,-87.725964,5
2,2492736,restaurant,risk_1_high,60636,2021-03-11,canvas,not_pass,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.779718,-87.654566,4
3,2492763,restaurant,risk_1_high,60657,2021-03-11,canvas,not_pass,na,41.933688,-87.644454,0
4,2492737,restaurant,risk_1_high,60647,2021-03-11,noninspection,not_pass,na,41.917557,-87.682664,0
...,...,...,...,...,...,...,...,...,...,...,...
217275,70269,restaurant,risk_1_high,60634,2010-01-04,license,pass,na,41.938443,-87.768318,0
217276,67757,restaurant,risk_2_medium,60601,2010-01-04,tag_removal,pass,na,41.884586,-87.631010,0
217277,67733,restaurant,risk_1_high,60613,2010-01-04,license,pass,na,41.961606,-87.675967,0
217279,52234,restaurant,risk_1_high,60657,2010-01-04,license,pass,na,41.938007,-87.644755,0


In [37]:
data.dtypes

inspection_id              object
facility_type              object
risk                       object
zip                        object
inspection_date    datetime64[ns]
inspection_type            object
results                    object
violations                 object
latitude                  float64
longitude                 float64
num_violations              int64
dtype: object

In [38]:
data['dow'] = data['inspection_date'].dt.day_name()
data['dow'] = data['dow'].str.lower()
data['month'] = data['inspection_date'].dt.month

In [39]:
data = data.astype({"month": 'str'})

In [40]:
data.head()

,inspection_id,facility_type,risk,zip,inspection_date,inspection_type,results,violations,latitude,longitude,num_violations,dow,month
0,2492768,restaurant,risk_2_medium,60647,2021-03-12,license,pass,na,41.917135,-87.687544,0,friday,3
1,2492746,daycare,risk_1_high,60624,2021-03-11,license,not_pass,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.885920,-87.725964,5,thursday,3
2,2492736,restaurant,risk_1_high,60636,2021-03-11,canvas,not_pass,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.779718,-87.654566,4,thursday,3
3,2492763,restaurant,risk_1_high,60657,2021-03-11,canvas,not_pass,na,41.933688,-87.644454,0,thursday,3
4,2492737,restaurant,risk_1_high,60647,2021-03-11,noninspection,not_pass,na,41.917557,-87.682664,0,thursday,3


## Step other

In [41]:
data['inspection_type'] =  data['inspection_type'].mask(data['inspection_type'].map(data['inspection_type'].value_counts(normalize=True)) < 0.0002, 'other')
data['facility_type'] =  data['facility_type'].mask(data['facility_type'].map(data['facility_type'].value_counts(normalize=True)) < 0.0002, 'other')
data['zip'] =  data['zip'].mask(data['zip'].map(data['zip'].value_counts(normalize=True)) < 0.002, 'other')

## Cambiar variable de respuesta a 0 y 1

In [43]:
data['results'] = data['results'].apply(lambda x: '1' if x in ['pass'] else '0' )

## Eliminar columnas que no nos sirven

In [44]:
data = data.drop(['inspection_id','inspection_date','violations'], axis=1)

data

,facility_type,risk,zip,inspection_type,results,latitude,longitude,num_violations,dow,month
0,restaurant,risk_2_medium,60647,license,0,41.917135,-87.687544,0,friday,3
1,daycare,risk_1_high,60624,license,0,41.885920,-87.725964,5,thursday,3
2,restaurant,risk_1_high,60636,canvas,0,41.779718,-87.654566,4,thursday,3
3,restaurant,risk_1_high,60657,canvas,0,41.933688,-87.644454,0,thursday,3
4,restaurant,risk_1_high,60647,noninspection,0,41.917557,-87.682664,0,thursday,3
...,...,...,...,...,...,...,...,...,...,...
217275,restaurant,risk_1_high,60634,license,0,41.938443,-87.768318,0,monday,1
217276,restaurant,risk_2_medium,60601,tag_removal,0,41.884586,-87.631010,0,monday,1
217277,restaurant,risk_1_high,60613,license,0,41.961606,-87.675967,0,monday,1
217279,restaurant,risk_1_high,60657,license,0,41.938007,-87.644755,0,monday,1


In [45]:
data.risk.value_counts()

risk_1_high      155156
risk_2_medium     42124
risk_3_low        19171
Name: risk, dtype: int64

In [47]:
data = pd.get_dummies(data, columns = ['facility_type', 'inspection_type','risk','zip', 'dow','month'])

data

,results,latitude,longitude,num_violations,facility_type_assisted_living,facility_type_bakery,facility_type_banquet,facility_type_bar,facility_type_butcher,facility_type_catering,...,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
0,0,41.917135,-87.687544,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,41.885920,-87.725964,5,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,41.779718,-87.654566,4,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,41.933688,-87.644454,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,41.917557,-87.682664,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217275,0,41.938443,-87.768318,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
217276,0,41.884586,-87.631010,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
217277,0,41.961606,-87.675967,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
217279,0,41.938007,-87.644755,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
def extract_violation_num(s) -> int:
    """
    Extrae el número de violación en un string. 
    :param s: string
              texto del registro
    :return: int
    """
    pattern = '\d+\.'  # numeros + punto
    result = re.findall(pattern, s)
    if not result:  # no hay violaciones
        return 0
    else:  # regresa número de violación
        return int(result[0].replace('.', ''))
    

def get_violations_incurred(s) -> list:
    """
    Extra el total de violaciones en un registro. 
    :param s: string
              Una celda de la columna 'violations'
    :return violation_nums: list
            lista con todas las infracciones, por ejemplo, [13,22,55]
    """
    all_violations = s.split('|')
    violation_nums = []
    for violation in all_violations:
        violation_nums.append(extract_violation_num(violation))
    
    return violation_nums
    
def add_extra_columns(df):
    """Añade columnas para one hot encoding."""
    for i in range(1, 80):
        column = 'violation_' + str(i)
        df[column] = 0
    
    return df

def add_one_hot_encoding(df):
    """ 
    Realiza todo el pipeline.
    
    :param df: pandas dataframe
    :return df: dataframe con el one hot encoding
    """
    
    # crear columnas
    df = add_extra_columns(df)
    df['lista_violaciones'] = df.violations.apply(get_violations_incurred)
    
    # luego llenamos df
    for i in range(len(df)):
        violaciones = df['lista_violaciones'][i]
        violaciones = [e for e in violaciones if e!= '0']
        for violacion in violaciones:
            column_index = violacion + 12
            df.iloc[i, column_index] = 1

    # tiramos columnas  temporales y las que están todas en ceros
    df.drop(columns=['lista_violaciones'], inplace = True)
    df = df.loc[(df.sum(axis=1) != 0), (df.sum(axis=0) != 0)]
    
    return df

In [ ]:
data_violations = add_one_hot_encoding(data)